### Schiebeparkplatz

[Lösungsidee](schiebeparkplatz.pdf)

<img src='schiebeparkplatz.png' width='300'>

Hinweis: das Programm muss eine in Bezug auf die Anzahl der zu verschiebenden Autos optimale Lösung finden, die Länge des Verschiebungswegs spielt keine Rolle: Ob ich 1 nach links oder 2 nach rechts gehe, ist gleich gut.

Die Beispieldaten von 0-5

In [35]:
nr = 1
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
print(f.read())

A N
4
O 1
P 3
Q 6
R 10



Lesen der ersten Eingabezeile. Damit wird die obere Wagenreihe modelliert

In [36]:
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
c1, c2 = f.readline().split()
oben = [chr(c) for c in range(ord(c1), ord(c2)+1)]    # obere Wagenreihe als Liste mit Buchstaben
oben
 

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']

In [37]:
n = int(f.readline())   # anzahl querstehender Wagen                 
quer = []               # Liste mit Namen und Position der querstehenden Wagen
for i in range(n):
    name, pos = f.readline().split()
    quer.append((name,int(pos)))
    
quer

[('O', 1), ('P', 3), ('Q', 6), ('R', 10)]

In [38]:
# Konstruktion des startstates als Tuple
startstate = ['.']*len(oben)
for c,k in quer:
    startstate[k]=c
    startstate[k+1]=c
    
startstate = tuple(startstate)
startstate

('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.')

In [39]:
# Zur Visualisierung der Situation
def show(state):
    print(*oben)
    print(*state)
    
show(startstate)

A B C D E F G H I J K L M N
. O O P P . Q Q . . R R . .


Zur Suche in dem Baum der Möglichkeiten benötigen wir die Funktionen *goaltest* und *nextstates*.

In [40]:
wagenIdx = 5    # globale Variable, Index des Wagens, der rausgefahren werden soll

def goaltest(state):
    return state[wagenIdx] == '.'

In [41]:
querwagen = [c[0] for c in quer]      # Liste mit den Buchstaben der querstehenden Wagen

def nextstates(state):
    tmp = []
    for c in querwagen:
        k = state.index(c)
        new_state = list(state)
        while k+2 < len(state) and state[k+2]=='.':   # solange rechts Platz ist
            new_state[k]='.'                          # verschiebe die beiden Zeichen in new_state                     
            new_state[k+2]=c                          # um eins nach rechts
            tmp.append(tuple(new_state))              # speichere den Zustand als möglichen Folgezustand
            k+=1
            
        k = state.index(c)                            
        new_state = list(state)
        while k-1 >= 0 and new_state[k-1]=='.':       # solange links Platz ist
            new_state[k-1]=c
            new_state[k+1]='.'
            tmp.append(tuple(new_state))
            k-=1
    return tmp

In [42]:

sorted(nextstates(startstate))
 

[('.', 'O', 'O', '.', 'P', 'P', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.'),
 ('.', 'O', 'O', 'P', 'P', '.', '.', '.', 'Q', 'Q', 'R', 'R', '.', '.'),
 ('.', 'O', 'O', 'P', 'P', '.', '.', 'Q', 'Q', '.', 'R', 'R', '.', '.'),
 ('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', '.', '.', 'R', 'R'),
 ('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', '.', 'R', 'R', '.'),
 ('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', 'R', 'R', '.', '.', '.'),
 ('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', 'R', 'R', '.', '.', '.', '.'),
 ('.', 'O', 'O', 'P', 'P', 'Q', 'Q', '.', '.', '.', 'R', 'R', '.', '.'),
 ('O', 'O', '.', 'P', 'P', '.', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.')]

In [43]:
from collections import deque
def bfs(startstate):
    frontier =  deque([startstate])
    prev = {startstate:None}
    if goaltest(startstate):
            return prev, startstate
    while frontier:
        state = frontier.popleft()
        for v in nextstates(state):
            if v not in prev:
                prev[v] = state
                if goaltest(v):
                    return prev, v
                frontier.append(v)

    

In [44]:
prev, goalstate = bfs(startstate)
goalstate

('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.')

In [45]:
prev

{('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.'): None}

In [46]:
def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path

In [47]:
path = reconstructPath(prev,goalstate)
path

[('.', 'O', 'O', 'P', 'P', '.', 'Q', 'Q', '.', '.', 'R', 'R', '.', '.')]

Um die Abfolge des Pfades in Worten wiederzugeben, definieren wir uns eine Funktion *schieb*

In [48]:
def schieb(s1, s2):
    for c in querwagen:
        i1 = s1.index(c)
        i2 = s2.index(c)
        if i1 > i2:
            return f'{c}: {i1-i2} nach links ' 
        if i1 < i2:
            return f'{c}: {i2-i1} nach rechts' 

In [49]:
for i in range(len(path)-1):
    print(schieb(path[i],path[i+1]))

In einer Schleife lassen ermitteln wir die Ergebnisse für alle Wagen

In [50]:
for w in oben:
    wagenIdx = ord(w)-ord('A')
    prev, state = bfs(startstate)
    path = reconstructPath(prev, state)
    print(w,': ',end='')
    for i in range(len(path)-1):
        print(schieb(path[i],path[i+1]),end='  ')
    print()

A : 
B : P: 1 nach rechts  O: 1 nach rechts  
C : O: 1 nach links   
D : P: 1 nach rechts  
E : O: 1 nach links   P: 1 nach links   
F : 
G : Q: 1 nach rechts  
H : Q: 2 nach rechts  
I : 
J : 
K : R: 1 nach rechts  
L : R: 2 nach rechts  
M : 
N : 


### Das vollständige Programm

In [51]:
from collections import deque

def goaltest(state):
    return state[wagenIdx] == '.'

def nextstates(state):
    tmp = []
    for c in querwagen:
        k = state.index(c)
        new_state = list(state)
        while k+2 < len(state) and state[k+2]=='.':   # solange rechts Platz ist
            new_state[k]='.'                          # verschiebe die beiden Zeichen in new_state                     
            new_state[k+2]=c                          # um eins nach rechts
            tmp.append(tuple(new_state))              # speichere den Zustand als möglichen Folgezustand
            k+=1
            
        k = state.index(c)                            
        new_state = list(state)
        while k-1 >= 0 and new_state[k-1]=='.':       # solange links Platz ist
            new_state[k-1]=c
            new_state[k+1]='.'
            tmp.append(tuple(new_state))
            k-=1
    return tmp

def bfs(startstate):
    frontier = deque([startstate])
    prev = {startstate:None}
    if goaltest(startstate):
            return prev, startstate
    while frontier:
        state = frontier.popleft()
        for v in nextstates(state):
            if v not in prev:
                prev[v] = state
                if goaltest(v):
                    return prev, v
                frontier.append(v)
                
def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path

def schieb(s1, s2):
    for c in querwagen:
        i1 = s1.index(c)
        i2 = s2.index(c)
        if i1 > i2:
            return f'{c}: {i1-i2} nach links ' 
        if i1 < i2:
            return f'{c}: {i2-i1} nach rechts' 

# Obere Wagenreihe        
nr = 3
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
c1, c2 = f.readline().split()
oben = [chr(c) for c in range(ord(c1), ord(c2)+1)]  

# Untere Wagenreihe
n = int(f.readline())   # anzahl querstehender Wagen                 
quer = []               # Liste mit Namen und Position der querstehenden Wagen
for i in range(n):
    name, pos = f.readline().split()
    quer.append((name,int(pos)))

querwagen = [c[0] for c in quer]   # Liste mit den Buchstaben der querstehenden Wagen

# Konstruktion startstate
startstate = ['.']*len(oben)
for c,k in quer:
    startstate[k]=c
    startstate[k+1]=c
startstate = tuple(startstate)

# Ausgabe Startstate
show(startstate)
print()

# Breitensuche und Ausgabe Ergebnisse

for w in oben:
    
    wagenIdx = ord(w)-ord('A')
    prev, state = bfs(startstate)
    path = reconstructPath(prev, state)
    print(w,': ',end='')
    for i in range(len(path)-1):
        print(schieb(path[i],path[i+1]),end='  ')
    print()

 

A B C D E F G H I J K L M N
. O O . P P . . Q Q R R S S

A : 
B : O: 1 nach rechts  
C : O: 1 nach links   
D : 
E : P: 1 nach rechts  
F : P: 2 nach rechts  
G : 
H : 
I : Q: 2 nach links   
J : Q: 1 nach links   
K : Q: 2 nach links   R: 2 nach links   
L : Q: 1 nach links   R: 1 nach links   
M : Q: 2 nach links   R: 2 nach links   S: 2 nach links   
N : Q: 1 nach links   R: 1 nach links   S: 1 nach links   
